In [ ]:
# Import relevant packages
import pymaid
import seaborn as sns
import json
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv
import sys
import CATMAID_CRED


In [ ]:
# Initialize CATMAID and load data
myInstance = CATMAID_CRED.CATMAID_CRED()
with open('demo_data.json', 'r') as f:
    json_neurons = json.load(f)

str_neurons = json.dumps(json_neurons)
nl = pymaid.json2neuron(str_neurons)

In [ ]:
# 1a. Plot all of the dbd neurons in red and a08a neurons in black.
# first, we need to get the index of the sensory neurons and the interneurons.  One way to do this is to index by if there is a soma present. 
sensory_index = [i for i,x in enumerate(nl.soma) if not x]
interneuron_index = [i for i,x in enumerate(nl.soma) if x]
sensory_neurons = nl[sensory_index]
interneurons = nl[interneuron_index]

fig,ax = pymaid.plot2d([sensory_neurons], connectors=False,linewidth=1.5, method='3d',color = 'r',alpha = .2,figsize = [15,15])
fig,ax = pymaid.plot2d([interneurons], ax = ax, connectors=False,linewidth=1.5, method='3d',color = 'k',alpha = .2,figsize = [15,15])

In [ ]:
# 1b. Plot a08as and dbds in black, and plot the synapses from dbds to to a08as in the same segment in green, and intersegmental connections in magenta.  Plot from a dorsal view.
# First, get the connectors between all a08a and dbd synapses using pymaid.get_connectors_between.  We have two segments, so four combinations of synapses.

cn_table_a1a1 = pymaid.get_connectors_between(sensory_neurons[0,3],interneurons[1,3],directional=True)
cn_table_a1a2 = pymaid.get_connectors_between(sensory_neurons[0,3],interneurons[0,2],directional=True)
cn_table_a2a2 = pymaid.get_connectors_between(sensory_neurons[1,2],interneurons[0,2],directional=True)
cn_table_a2a1 = pymaid.get_connectors_between(sensory_neurons[1,2],interneurons[1,3],directional=True)

# First plot dbd and a08a neurons.
fig,ax = pymaid.plot2d([sensory_neurons], connectors=False,linewidth=1.5, method='3d',color = 'k',alpha = .2,figsize = [15,15])
fig,ax = pymaid.plot2d([interneurons], ax = ax, connectors=False,linewidth=1.5, method='3d',color = 'k',alpha = .2,figsize = [15,15])

# Next plot the common synapses on top. Hint: you need to figure out how to convert a dataframe column from a list to an array. Use a for loop to put each coordinate in a list, then conver it to an array.
# You can use for loops to restructure. 
#     conloc_a1_a1 = list()
#     for i in range(len(cn_table_a1a1.connector_loc)):
#         conloc_a1_a1.append(cn_table_a1a1.connector_loc[i])

conloc_a1a1 = cn_table_a1a1.connector_loc.values
conloc_a1a1 = [val for sublist in conloc_a1a1 for val in conloc_a1a1]
conloc_a2a2 = cn_table_a2a2.connector_loc.values
conloc_a2a2 = [val for sublist in conloc_a2a2 for val in conloc_a2a2]
conloc_same = np.concatenate([np.array(conloc_a1a1),np.array(conloc_a2a2)])

conloc_a1a2 = cn_table_a1a2.connector_loc.values
conloc_a1a2  = [val for sublist in conloc_a1a2 for val in conloc_a1a2]
conloc_a2a1 = cn_table_a2a1.connector_loc.values
conloc_a2a1 = [val for sublist in conloc_a2a1 for val in conloc_a2a1]
conloc_inter = np.concatenate([np.array(conloc_a2a1),np.array(conloc_a2a1)])

fig,ax = pymaid.plot2d(np.array(conloc_same), method='3d', ax=ax,
                       scatter_kws={'s': 20,
                                    'color': 'g',
                                    'marker': 'D'})

fig,ax = pymaid.plot2d(np.array(conloc_inter), method='3d', ax=ax,
                       scatter_kws={'s': 20,
                                    'color': 'm',
                                    'marker': 'D'})
                       
ax.elev = 90



array([[ 49806.6,  74259.6, 122600. ],
       [ 49806.6,  74259.6, 122600. ],
       [ 50631.2,  73233.6, 120900. ],
       [ 50608.4,  73195.6, 121050. ],
       [ 47766. ,  75517.4, 122700. ],
       [ 48951.6,  75040.5, 123300. ],
       [ 48951.6,  75040.5, 123300. ],
       [ 49297.4,  76501.6, 126250. ],
       [ 49145.4,  74692.8, 122850. ],
       [ 49436.1,  74654.8, 123150. ],
       [ 49211.9,  74696.6, 123450. ],
       [ 49211.9,  74696.6, 123450. ],
       [ 49559.6,  74421.1, 122600. ],
       [ 62453. ,  78306.6, 124250. ],
       [ 62169.9,  77683.4, 123450. ],
       [ 61499.2,  76562.4, 122250. ],
       [ 62377. ,  77862. , 122900. ],
       [ 62107.2,  78667.6, 125400. ],
       [ 61689.2,  78278.1, 125950. ],
       [ 62031.2,  78741.7, 125700. ],
       [ 62031.2,  78082.4, 124600. ],
       [ 62892.8,  78389.3, 123050. ],
       [ 62149. ,  79095.1, 124950. ]])

In [ ]:
#1c.  It looks like the dbd in the a2 segment has synapses onto both the A1 and A2 A08a neurons.  Get the connectivity between dbd_a2 and a08a a1/2.
dbd = nl[2,3]
a08a = nl[6,7,4,1]

neuron_adj = pymaid.adjacency_matrix(dbd,a08a)
g = sns.heatmap(neuron_adj, square=True)
g.set_yticklabels(dbd.neuron_name, rotation=0, fontsize=14)
g.set_xticklabels(a08a.neuron_name, rotation=90, fontsize=14)
plt.show()

In [ ]:
# 2. Cut out just the lateral and medial dendrites of a08a_a1r. Hint: The sensory neuron vbd makes synapses onto the lateral arbor.  You want to cut at branch points.  Use the functions pymaid.classify_nodes to find branch points and pymaid.dist_between
# to measure distances between treenodes.

#Get all partners for a08a_a1r
neuron = a08a[1]
table = pymaid.get_partners(neuron,threshold=3)

#Get the connector details for synapses between dbd-a08a and vbd-a08a
cn_table_dbd = pymaid.get_connectors_between(table.skeleton_id[0],neuron)
cn_table_vbd = pymaid.get_connectors_between(table.skeleton_id[7],neuron)

# Classify nodes to get branch indices as well as strahler order
neuron = pymaid.classify_nodes(neuron,inplace=False)
neuron = pymaid.strahler_index(neuron,inplace=False,min_twig_size=2)

# Find the indices for branches and the indices for the backbone.  Use strahler orders > 3 for backbones.
branch_index = neuron.nodes.index[neuron.nodes.type == 'branch']
backbone_index = neuron.nodes.index[neuron.nodes.strahler_index > 3]
branch_backbone_treenodes = np.array(neuron.nodes.treenode_id.loc[branch_index.intersection(backbone_index)])





In [ ]:
# Calculate the distances between a dbd and vbd synapse to all treenodes that are branches from the backbone
dist_vbd = []
dist_dbd = []
for i in range(len(branch_backbone_treenodes)):
    
    dist_vbd.append(pymaid.dist_between(neuron,cn_table_vbd.treenode2_id[0],branch_backbone_treenodes[i]))
    dist_dbd.append(pymaid.dist_between(neuron,cn_table_dbd.treenode2_id[0],branch_backbone_treenodes[i]))
    
dist_vbd = np.array(dist_vbd)
dist_dbd = np.array(dist_dbd)

# The index for the cutpoint for the lateral dendrite is the branch closest to a vbd synapse.
# The index for the cutpoint for the medial dendrite is the branch closest to a dbd synapse.
lateral_index = np.where(dist_vbd == min(dist_vbd))
medial_index = np.where(dist_dbd == min(dist_dbd))
lateral_branch = [branch_backbone_treenodes[np.asscalar(lateral_index[0])]]
medial_branch = [branch_backbone_treenodes[np.asscalar(medial_index[0])]]

# Cut the neuron at the branch points, and keep everything distal to the cut point.
lateral_dendrite = pymaid.cut_neuron(neuron,lateral_branch,ret='distal')
medial_dendrite = pymaid.cut_neuron(neuron,medial_branch,ret='distal')

In [ ]:
# Plot the neuron, and the connectors from the lateral dendrite in green and the medial dendrite in magenta.

fig,ax = pymaid.plot2d([neuron], connectors=False,linewidth=1.5, method='3d',color = 'k',alpha = .2,figsize = [15,15])
fig,ax = pymaid.plot2d(lateral_dendrite,connectors=False,linewidth=2,method='3d',color='g',alpha=.2,ax=ax)
fig,ax = pymaid.plot2d(medial_dendrite,connectors=False,linewidth=2,method='3d',color='m',alpha=.2,ax=ax)


fig,ax = pymaid.plot2d(lateral_dendrite.connectors[['x','y','z']].values, method='3d', ax=ax,
                       scatter_kws={'s': 20,
                                    'color': 'g',
                                    'marker': 'D'})

fig,ax = pymaid.plot2d(medial_dendrite.connectors[['x','y','z']].values, method='3d', ax=ax,
                       scatter_kws={'s': 20,
                                    'color': 'm',
                                    'marker': 'D'})
                       
